<a href="https://colab.research.google.com/github/DomStue/Coursera_Capstone/blob/master/Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Capstone Project**
This project is mainly used to work on the capstone project of the data science course of IBM which I am attending

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [ ]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


**Import of the dataset:**

In [ ]:
!wget -O Data-Collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

--2020-09-14 17:17:11--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘Data-Collisions.csv’

Data-Collisions.csv 100%[===================>]  70.49M  19.9MB/s    in 4.9s    

2020-09-14 17:17:17 (14.5 MB/s) - ‘Data-Collisions.csv’ saved [73917638/73917638]



In [ ]:
df = pd.read_csv('Data-Collisions.csv')
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,,NaN,1,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [ ]:
df.shape

(194673, 38)

#Selecting only a few features from the dataset
Note: The goal was to achieve a general model (not seattle exclusively). Although there is some interesting data about car crash hot spots, seattle specific data like street names or coordinates are excluded

In [ ]:
Features = df[['ADDRTYPE','SEVERITYDESC','COLLISIONTYPE','PERSONCOUNT','PEDCOUNT','PEDCYLCOUNT','VEHCOUNT','INCDTTM','JUNCTIONTYPE','INATTENTIONIND','UNDERINFL','WEATHER', 'LIGHTCOND', 'ROADCOND','PEDROWNOTGRNT','SPEEDING','SEGLANEKEY','CROSSWALKKEY','HITPARKEDCAR']]
Features

,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDTTM,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,LIGHTCOND,ROADCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,Intersection,Injury Collision,Angles,2,0,0,2,3/27/2013 2:54:00 PM,At Intersection (intersection related),NaN,N,Overcast,Daylight,Wet,NaN,NaN,0,0,N
1,Block,Property Damage Only Collision,Sideswipe,2,0,0,2,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),NaN,0,Raining,Dark - Street Lights On,Wet,NaN,NaN,0,0,N
2,Block,Property Damage Only Collision,Parked Car,4,0,0,3,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),NaN,0,Overcast,Daylight,Dry,NaN,NaN,0,0,N
3,Block,Property Damage Only Collision,Other,3,0,0,3,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),NaN,N,Clear,Daylight,Dry,NaN,NaN,0,0,N
4,Intersection,Injury Collision,Angles,2,0,0,2,1/28/2004 8:04:00 AM,At Intersection (intersection related),NaN,0,Raining,Daylight,Wet,NaN,NaN,0,0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194668,Block,Injury Collision,Head On,3,0,0,2,11/12/2018 8:12:00 AM,Mid-Block (not related to intersection),NaN,N,Clear,Daylight,Dry,NaN,NaN,0,0,N
194669,Block,Property Damage Only Collision,Rear Ended,2,0,0,2,12/18/2018 9:14:00 AM,Mid-Block (not related to intersection),Y,N,Raining,Daylight,Wet,NaN,NaN,0,0,N
194670,Intersection,Injury Collision,Left Turn,3,0,0,2,1/19/2019 9:25:00 AM,At Intersection (intersection related),NaN,N,Clear,Daylight,Dry,NaN,NaN,0,0,N
194671,Intersection,Injury Collision,Cycles,2,0,1,1,1/15/2019 4:48:00 PM,At Intersection (intersection related),NaN,N,Clear,Dusk,Dry,NaN,NaN,4308,0,N


**Change date format and add weekdays and hour of day**

In [ ]:
Features['INCDTTM'] = pd.to_datetime(Features['INCDTTM'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Features['WEEKDAY'] = Features['INCDTTM'].dt.dayofweek
Features['HOURDAY'] = Features['INCDTTM'].dt.hour
Features.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDTTM,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,LIGHTCOND,ROADCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEEKDAY,HOURDAY
0,Intersection,Injury Collision,Angles,2,0,0,2,2013-03-27 14:54:00,At Intersection (intersection related),NaN,N,Overcast,Daylight,Wet,NaN,NaN,0,0,N,2,14
1,Block,Property Damage Only Collision,Sideswipe,2,0,0,2,2006-12-20 18:55:00,Mid-Block (not related to intersection),NaN,0,Raining,Dark - Street Lights On,Wet,NaN,NaN,0,0,N,2,18
2,Block,Property Damage Only Collision,Parked Car,4,0,0,3,2004-11-18 10:20:00,Mid-Block (not related to intersection),NaN,0,Overcast,Daylight,Dry,NaN,NaN,0,0,N,3,10
3,Block,Property Damage Only Collision,Other,3,0,0,3,2013-03-29 09:26:00,Mid-Block (not related to intersection),NaN,N,Clear,Daylight,Dry,NaN,NaN,0,0,N,4,9
4,Intersection,Injury Collision,Angles,2,0,0,2,2004-01-28 08:04:00,At Intersection (intersection related),NaN,0,Raining,Daylight,Wet,NaN,NaN,0,0,N,2,8


**Drop the date format**

In [ ]:
Features = Features.drop('INCDTTM', axis=1)
Features.head()

,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,LIGHTCOND,ROADCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEEKDAY,HOURDAY
0,Intersection,Injury Collision,Angles,2,0,0,2,At Intersection (intersection related),NaN,N,Overcast,Daylight,Wet,NaN,NaN,0,0,N,2,14
1,Block,Property Damage Only Collision,Sideswipe,2,0,0,2,Mid-Block (not related to intersection),NaN,0,Raining,Dark - Street Lights On,Wet,NaN,NaN,0,0,N,2,18
2,Block,Property Damage Only Collision,Parked Car,4,0,0,3,Mid-Block (not related to intersection),NaN,0,Overcast,Daylight,Dry,NaN,NaN,0,0,N,3,10
3,Block,Property Damage Only Collision,Other,3,0,0,3,Mid-Block (not related to intersection),NaN,N,Clear,Daylight,Dry,NaN,NaN,0,0,N,4,9
4,Intersection,Injury Collision,Angles,2,0,0,2,At Intersection (intersection related),NaN,0,Raining,Daylight,Wet,NaN,NaN,0,0,N,2,8


**Investigate features with empty entries**

NaN are filled with No, Unknown and Other values. Unknown and Other are values are combined in each feature





In [ ]:
Features.isnull().sum()

ADDRTYPE            1926
SEVERITYDESC           0
COLLISIONTYPE       4904
PERSONCOUNT            0
PEDCOUNT               0
PEDCYLCOUNT            0
VEHCOUNT               0
JUNCTIONTYPE        6329
INATTENTIONIND    164868
UNDERINFL           4884
WEATHER             5081
LIGHTCOND           5170
ROADCOND            5012
PEDROWNOTGRNT     190006
SPEEDING          185340
SEGLANEKEY             0
CROSSWALKKEY           0
HITPARKEDCAR           0
WEEKDAY                0
HOURDAY                0
dtype: int64

In [ ]:
Features["ADDRTYPE"].fillna("No", inplace = True)
Features['ADDRTYPE'].value_counts(dropna=False)

Block           126926
Intersection     65070
No                1926
Alley              751
Name: ADDRTYPE, dtype: int64

In [ ]:
Features['SEVERITYDESC'].value_counts(dropna=False)

Property Damage Only Collision    136485
Injury Collision                   58188
Name: SEVERITYDESC, dtype: int64

In [ ]:
Features['COLLISIONTYPE'].fillna("Other", inplace = True)
Features['COLLISIONTYPE'].value_counts(dropna=False)

Parked Car    47987
Angles        34674
Rear Ended    34090
Other         28607
Sideswipe     18609
Left Turn     13703
Pedestrian     6608
Cycles         5415
Right Turn     2956
Head On        2024
Name: COLLISIONTYPE, dtype: int64

In [ ]:
Features['JUNCTIONTYPE'].fillna("Unknown", inplace = True)
Features['JUNCTIONTYPE'].value_counts(dropna=False)

Mid-Block (not related to intersection)              89800
At Intersection (intersection related)               62810
Mid-Block (but intersection related)                 22790
Driveway Junction                                    10671
Unknown                                               6338
At Intersection (but not related to intersection)     2098
Ramp Junction                                          166
Name: JUNCTIONTYPE, dtype: int64

In [ ]:
Features['INATTENTIONIND'].fillna("N", inplace = True)
Features['UNDERINFL'].replace(to_replace=['N','Y'], value=[0,1],inplace=True)
Features['INATTENTIONIND'].value_counts(dropna=False)

N    164868
Y     29805
Name: INATTENTIONIND, dtype: int64

In [ ]:
Features['UNDERINFL'].fillna("N", inplace = True)
Features['UNDERINFL'].replace(to_replace=['N','Y'], value=[0,1],inplace=True)
Features['UNDERINFL'].replace(to_replace=['0','1'], value=[0,1],inplace=True)
Features['UNDERINFL'].value_counts(dropna=False)

0    185552
1      9121
Name: UNDERINFL, dtype: int64

In [ ]:
Features['UNDERINFL'].value_counts(dropna=False)

0    185552
1      9121
Name: UNDERINFL, dtype: int64

In [ ]:
Features['WEATHER'].fillna("Unknown", inplace = True)
Features['WEATHER'].replace(to_replace='Other', value='Unknown',inplace=True)
Features['WEATHER'].value_counts(dropna=False)

Clear                       111135
Raining                      33145
Overcast                     27714
Unknown                      21004
Snowing                        907
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [ ]:
Features['LIGHTCOND'].fillna("Unknown", inplace = True)
Features['LIGHTCOND'].replace(to_replace='Other', value='Unknown',inplace=True)
Features['LIGHTCOND'].value_counts(dropna=False)

Daylight                    116137
Dark - Street Lights On      48507
Unknown                      18878
Dusk                          5902
Dawn                          2502
Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

In [ ]:
Features['ROADCOND'].fillna("Unknown", inplace = True)
Features['ROADCOND'].replace(to_replace='Other', value='Unknown',inplace=True)
Features['ROADCOND'].value_counts(dropna=False)

Dry               124510
Wet                47474
Unknown            20222
Ice                 1209
Snow/Slush          1004
Standing Water       115
Sand/Mud/Dirt         75
Oil                   64
Name: ROADCOND, dtype: int64

In [ ]:
Features['PEDROWNOTGRNT'].fillna("N", inplace = True)
Features['PEDROWNOTGRNT'].value_counts(dropna=False)

N    190006
Y      4667
Name: PEDROWNOTGRNT, dtype: int64

In [ ]:
Features['SPEEDING'].fillna("N", inplace = True)
Features['SPEEDING'].value_counts(dropna=False)

N    185340
Y      9333
Name: SPEEDING, dtype: int64

In [ ]:
Features.isnull().sum()

ADDRTYPE          0
SEVERITYDESC      0
COLLISIONTYPE     0
PERSONCOUNT       0
PEDCOUNT          0
PEDCYLCOUNT       0
VEHCOUNT          0
JUNCTIONTYPE      0
INATTENTIONIND    0
UNDERINFL         0
WEATHER           0
LIGHTCOND         0
ROADCOND          0
PEDROWNOTGRNT     0
SPEEDING          0
SEGLANEKEY        0
CROSSWALKKEY      0
HITPARKEDCAR      0
WEEKDAY           0
HOURDAY           0
dtype: int64

In [ ]:
Features.head()

,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,LIGHTCOND,ROADCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEEKDAY,HOURDAY
0,Intersection,Injury Collision,Angles,2,0,0,2,At Intersection (intersection related),N,0,Overcast,Daylight,Wet,N,N,0,0,N,2,14
1,Block,Property Damage Only Collision,Sideswipe,2,0,0,2,Mid-Block (not related to intersection),N,0,Raining,Dark - Street Lights On,Wet,N,N,0,0,N,2,18
2,Block,Property Damage Only Collision,Parked Car,4,0,0,3,Mid-Block (not related to intersection),N,0,Overcast,Daylight,Dry,N,N,0,0,N,3,10
3,Block,Property Damage Only Collision,Other,3,0,0,3,Mid-Block (not related to intersection),N,0,Clear,Daylight,Dry,N,N,0,0,N,4,9
4,Intersection,Injury Collision,Angles,2,0,0,2,At Intersection (intersection related),N,0,Raining,Daylight,Wet,N,N,0,0,N,2,8


**After filling empty values we start to one-hot-encode the data**

In [ ]:
Features_enc = Features
Features_enc.head()

,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,LIGHTCOND,ROADCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEEKDAY,HOURDAY
0,Intersection,Injury Collision,Angles,2,0,0,2,At Intersection (intersection related),N,0,Overcast,Daylight,Wet,N,N,0,0,N,2,14
1,Block,Property Damage Only Collision,Sideswipe,2,0,0,2,Mid-Block (not related to intersection),N,0,Raining,Dark - Street Lights On,Wet,N,N,0,0,N,2,18
2,Block,Property Damage Only Collision,Parked Car,4,0,0,3,Mid-Block (not related to intersection),N,0,Overcast,Daylight,Dry,N,N,0,0,N,3,10
3,Block,Property Damage Only Collision,Other,3,0,0,3,Mid-Block (not related to intersection),N,0,Clear,Daylight,Dry,N,N,0,0,N,4,9
4,Intersection,Injury Collision,Angles,2,0,0,2,At Intersection (intersection related),N,0,Raining,Daylight,Wet,N,N,0,0,N,2,8


In [ ]:
#Var = enc.fit_transform(Features_enc)

In [ ]:
Y = df['SEVERITYCODE']
Y.head()

0    2
1    1
2    1
3    1
4    2
Name: SEVERITYCODE, dtype: int64

In [ ]:
#Var =  preprocessing.StandardScaler().fit(Var).transform(Var)

In [ ]:
Features_enc = pd.concat((Features_enc,pd.get_dummies(Features_enc.ADDRTYPE)),axis=1)
Features_enc.drop(['No'], axis = 1,inplace=True)
Features_enc.drop(['ADDRTYPE'], axis = 1,inplace=True)

In [ ]:
Features_enc = pd.concat((Features_enc,pd.get_dummies(Features_enc.SEVERITYDESC)),axis=1)
Features_enc.drop(['Property Damage Only Collision'], axis = 1,inplace=True)
Features_enc.drop(['SEVERITYDESC'], axis = 1,inplace=True)
Features_enc.head()

,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,LIGHTCOND,ROADCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEEKDAY,HOURDAY,Alley,Block,Intersection,Injury Collision
0,Angles,2,0,0,2,At Intersection (intersection related),N,0,Overcast,Daylight,Wet,N,N,0,0,N,2,14,0,0,1,1
1,Sideswipe,2,0,0,2,Mid-Block (not related to intersection),N,0,Raining,Dark - Street Lights On,Wet,N,N,0,0,N,2,18,0,1,0,0
2,Parked Car,4,0,0,3,Mid-Block (not related to intersection),N,0,Overcast,Daylight,Dry,N,N,0,0,N,3,10,0,1,0,0
3,Other,3,0,0,3,Mid-Block (not related to intersection),N,0,Clear,Daylight,Dry,N,N,0,0,N,4,9,0,1,0,0
4,Angles,2,0,0,2,At Intersection (intersection related),N,0,Raining,Daylight,Wet,N,N,0,0,N,2,8,0,0,1,1


In [ ]:
Features_enc = pd.concat((Features_enc,pd.get_dummies(Features_enc.COLLISIONTYPE)),axis=1)
Features_enc.drop(['Other'], axis = 1,inplace=True)
Features_enc.drop(['COLLISIONTYPE'], axis = 1,inplace=True)
Features_enc.head()

,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,LIGHTCOND,ROADCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEEKDAY,HOURDAY,Alley,Block,Intersection,Injury Collision,Angles,Cycles,Head On,Left Turn,Parked Car,Pedestrian,Rear Ended,Right Turn,Sideswipe
0,2,0,0,2,At Intersection (intersection related),N,0,Overcast,Daylight,Wet,N,N,0,0,N,2,14,0,0,1,1,1,0,0,0,0,0,0,0,0
1,2,0,0,2,Mid-Block (not related to intersection),N,0,Raining,Dark - Street Lights On,Wet,N,N,0,0,N,2,18,0,1,0,0,0,0,0,0,0,0,0,0,1
2,4,0,0,3,Mid-Block (not related to intersection),N,0,Overcast,Daylight,Dry,N,N,0,0,N,3,10,0,1,0,0,0,0,0,0,1,0,0,0,0
3,3,0,0,3,Mid-Block (not related to intersection),N,0,Clear,Daylight,Dry,N,N,0,0,N,4,9,0,1,0,0,0,0,0,0,0,0,0,0,0
4,2,0,0,2,At Intersection (intersection related),N,0,Raining,Daylight,Wet,N,N,0,0,N,2,8,0,0,1,1,1,0,0,0,0,0,0,0,0


In [ ]:
Features_enc = pd.concat((Features_enc,pd.get_dummies(Features_enc.JUNCTIONTYPE)),axis=1)
Features_enc.drop(['Unknown'], axis = 1,inplace=True)
Features_enc.drop(['JUNCTIONTYPE'], axis = 1,inplace=True)
Features_enc.head()

,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INATTENTIONIND,UNDERINFL,WEATHER,LIGHTCOND,ROADCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEEKDAY,HOURDAY,Alley,Block,Intersection,Injury Collision,Angles,Cycles,Head On,Left Turn,Parked Car,Pedestrian,Rear Ended,Right Turn,Sideswipe,At Intersection (but not related to intersection),At Intersection (intersection related),Driveway Junction,Mid-Block (but intersection related),Mid-Block (not related to intersection),Ramp Junction
0,2,0,0,2,N,0,Overcast,Daylight,Wet,N,N,0,0,N,2,14,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,2,0,0,2,N,0,Raining,Dark - Street Lights On,Wet,N,N,0,0,N,2,18,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,4,0,0,3,N,0,Overcast,Daylight,Dry,N,N,0,0,N,3,10,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,3,0,0,3,N,0,Clear,Daylight,Dry,N,N,0,0,N,4,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,2,0,0,2,N,0,Raining,Daylight,Wet,N,N,0,0,N,2,8,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
Features_enc = pd.concat((Features_enc,pd.get_dummies(Features_enc.INATTENTIONIND)),axis=1)
Features_enc.drop(['N'], axis = 1,inplace=True)
Features_enc.drop(['INATTENTIONIND'], axis = 1,inplace=True)
Features_enc.head()

,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,UNDERINFL,WEATHER,LIGHTCOND,ROADCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEEKDAY,HOURDAY,Alley,Block,Intersection,Injury Collision,Angles,Cycles,Head On,Left Turn,Parked Car,Pedestrian,Rear Ended,Right Turn,Sideswipe,At Intersection (but not related to intersection),At Intersection (intersection related),Driveway Junction,Mid-Block (but intersection related),Mid-Block (not related to intersection),Ramp Junction,Y
0,2,0,0,2,0,Overcast,Daylight,Wet,N,N,0,0,N,2,14,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,2,0,0,2,0,Raining,Dark - Street Lights On,Wet,N,N,0,0,N,2,18,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
2,4,0,0,3,0,Overcast,Daylight,Dry,N,N,0,0,N,3,10,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3,3,0,0,3,0,Clear,Daylight,Dry,N,N,0,0,N,4,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,2,0,0,2,0,Raining,Daylight,Wet,N,N,0,0,N,2,8,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
Features_enc = pd.concat((Features_enc,pd.get_dummies(Features_enc.WEATHER)),axis=1)
Features_enc.drop(['Unknown'], axis = 1,inplace=True)
Features_enc.drop(['WEATHER'], axis = 1,inplace=True)
Features_enc.head()

,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,UNDERINFL,LIGHTCOND,ROADCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEEKDAY,HOURDAY,Alley,Block,Intersection,Injury Collision,Angles,Cycles,Head On,Left Turn,Parked Car,Pedestrian,Rear Ended,Right Turn,Sideswipe,At Intersection (but not related to intersection),At Intersection (intersection related),Driveway Junction,Mid-Block (but intersection related),Mid-Block (not related to intersection),Ramp Junction,Y,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Overcast,Partly Cloudy,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing
0,2,0,0,2,0,Daylight,Wet,N,N,0,0,N,2,14,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,2,0,0,2,0,Dark - Street Lights On,Wet,N,N,0,0,N,2,18,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
2,4,0,0,3,0,Daylight,Dry,N,N,0,0,N,3,10,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,3,0,0,3,0,Daylight,Dry,N,N,0,0,N,4,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
4,2,0,0,2,0,Daylight,Wet,N,N,0,0,N,2,8,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
Features_enc = pd.concat((Features_enc,pd.get_dummies(Features_enc.LIGHTCOND)),axis=1)
Features_enc.drop(['Unknown'], axis = 1,inplace=True)
Features_enc.drop(['LIGHTCOND'], axis = 1,inplace=True)
Features_enc.head()

,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,UNDERINFL,ROADCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEEKDAY,HOURDAY,Alley,Block,Intersection,Injury Collision,Angles,Cycles,Head On,Left Turn,Parked Car,Pedestrian,Rear Ended,Right Turn,Sideswipe,At Intersection (but not related to intersection),At Intersection (intersection related),Driveway Junction,Mid-Block (but intersection related),Mid-Block (not related to intersection),Ramp Junction,Y,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Overcast,Partly Cloudy,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk
0,2,0,0,2,0,Wet,N,N,0,0,N,2,14,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1,2,0,0,2,0,Wet,N,N,0,0,N,2,18,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,4,0,0,3,0,Dry,N,N,0,0,N,3,10,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
3,3,0,0,3,0,Dry,N,N,0,0,N,4,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,2,0,0,2,0,Wet,N,N,0,0,N,2,8,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [ ]:
Features_enc = pd.concat((Features_enc,pd.get_dummies(Features_enc.ROADCOND)),axis=1)
Features_enc.drop(['Unknown'], axis = 1,inplace=True)
Features_enc.drop(['ROADCOND'], axis = 1,inplace=True)
Features_enc.head()

,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,UNDERINFL,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEEKDAY,HOURDAY,Alley,Block,Intersection,Injury Collision,Angles,Cycles,Head On,Left Turn,Parked Car,Pedestrian,Rear Ended,Right Turn,Sideswipe,At Intersection (but not related to intersection),At Intersection (intersection related),Driveway Junction,Mid-Block (but intersection related),Mid-Block (not related to intersection),Ramp Junction,Y,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Overcast,Partly Cloudy,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk,Dry,Ice,Oil,Sand/Mud/Dirt,Snow/Slush,Standing Water,Wet
0,2,0,0,2,0,N,N,0,0,N,2,14,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,2,0,0,2,0,N,N,0,0,N,2,18,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2,4,0,0,3,0,N,N,0,0,N,3,10,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
3,3,0,0,3,0,N,N,0,0,N,4,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,2,0,0,2,0,N,N,0,0,N,2,8,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [ ]:
Features_enc['PEDROWNOTGRNT'].replace(to_replace=['N','Y'], value=[0,1],inplace=True)
Features_enc['SPEEDING'].replace(to_replace=['N','Y'], value=[0,1],inplace=True)
Features_enc['HITPARKEDCAR'].replace(to_replace=['N','Y'], value=[0,1],inplace=True)

In [ ]:
Features_enc.head()

,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,UNDERINFL,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEEKDAY,HOURDAY,Alley,Block,Intersection,Injury Collision,Angles,Cycles,Head On,Left Turn,Parked Car,Pedestrian,Rear Ended,Right Turn,Sideswipe,At Intersection (but not related to intersection),At Intersection (intersection related),Driveway Junction,Mid-Block (but intersection related),Mid-Block (not related to intersection),Ramp Junction,Y,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Overcast,Partly Cloudy,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk,Dry,Ice,Oil,Sand/Mud/Dirt,Snow/Slush,Standing Water,Wet
0,2,0,0,2,0,0,0,0,0,0,2,14,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,2,0,0,2,0,0,0,0,0,0,2,18,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2,4,0,0,3,0,0,0,0,0,0,3,10,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
3,3,0,0,3,0,0,0,0,0,0,4,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,2,0,0,2,0,0,0,0,0,0,2,8,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


**In the next section we apply feature selection to reduce our dataset to only 20 features with the highest impact**

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(Features_enc,Y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(Features_enc.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(20,'Score'))  #print 10 best features

                                      Specs         Score
8                              CROSSWALKKEY  3.186417e+09
7                                SEGLANEKEY  8.638549e+07
15                         Injury Collision  1.364850e+05
20                               Parked Car  1.356930e+04
1                                  PEDCOUNT  1.248884e+04
21                               Pedestrian  1.132928e+04
2                               PEDCYLCOUNT  8.818806e+03
17                                   Cycles  8.608345e+03
5                             PEDROWNOTGRNT  8.085270e+03
26   At Intersection (intersection related)  5.360734e+03
14                             Intersection  5.137017e+03
29  Mid-Block (not related to intersection)  2.939348e+03
22                               Rear Ended  2.811283e+03
0                               PERSONCOUNT  2.473877e+03
24                                Sideswipe  2.395238e+03
13                                    Block  2.312244e+03
9             

In [ ]:
Features_enc_new = SelectKBest(chi2, k=10).fit_transform(Features_enc, Y)
Features_enc_new

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
X = preprocessing.StandardScaler().fit(Features_enc_new).transform(Features_enc_new)
X[0:5]

array([[-0.18743029, -0.16958841, -0.1567239 , -0.08124849, -0.13536153,
         1.53153084, -0.16915004, -0.57196238, -0.18744809,  1.44892892],
       [-0.18743029, -0.16958841, -0.1567239 , -0.08124849, -0.13536153,
        -0.65294147, -0.16915004, -0.57196238, -0.18744809, -0.69016498],
       [-0.18743029, -0.16958841, -0.1567239 , -0.08124849, -0.13536153,
        -0.65294147, -0.16915004,  1.74836673, -0.18744809, -0.69016498],
       [-0.18743029, -0.16958841, -0.1567239 , -0.08124849, -0.13536153,
        -0.65294147, -0.16915004, -0.57196238, -0.18744809, -0.69016498],
       [-0.18743029, -0.16958841, -0.1567239 , -0.08124849, -0.13536153,
         1.53153084, -0.16915004, -0.57196238, -0.18744809,  1.44892892]])

In [ ]:
print ('Train set:', X.shape,  Y.shape)

Train set: (194673, 10) (194673,)


**Now the machine learning of the model starts**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k = 7
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
yhat = neigh.predict(X_test)

In [ ]:
from sklearn import metrics
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat))

Test set Accuracy:  0.9989727786337956


In [ ]:
from sklearn.tree import DecisionTreeClassifier
#X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)
DTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
DTree # it shows the default parameters

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
DTree.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
predTree = DTree.predict(X_test)

In [ ]:
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, predTree))

DecisionTrees's Accuracy:  1.0


In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
yhat_log = clf.predict(X_test)

In [ ]:
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat_log))

Test set Accuracy:  0.9994863893168978


In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
LR

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
yhat_svm = LR.predict(X_test)

In [ ]:
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat_svm))

Test set Accuracy:  1.0


In [ ]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [ ]:
knn_acc = metrics.accuracy_score(y_test, yhat)
Tree_acc = metrics.accuracy_score(y_test, predTree)
svm_acc = metrics.accuracy_score(y_test, yhat_log)
log_acc = metrics.accuracy_score(y_test, yhat_svm)

In [ ]:
knn_jacc = metrics.jaccard_similarity_score(y_test, yhat)
Tree_jacc = metrics.jaccard_similarity_score(y_test, predTree)
svm_jacc = metrics.jaccard_similarity_score(y_test, yhat_log)
log_jacc = metrics.jaccard_similarity_score(y_test, yhat_svm)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


In [ ]:
knn_f1 = metrics.f1_score(y_test, yhat, average='weighted')
Tree_f1 = metrics.f1_score(y_test, predTree, average='weighted')
svm_f1 = metrics.f1_score(y_test, yhat_log, average='weighted')
log_f1 = metrics.f1_score(y_test, yhat_svm, average='weighted')

In [ ]:
report = pd.DataFrame(columns = ['Algorithm','Accuracy','Jaccard','F1-score'])

knn_row = {'Algorithm':'KNN', 'Accuracy':knn_acc, 'Jaccard':knn_jacc, 'F1-score':knn_f1}
report = report.append(knn_row, ignore_index=True)

Tree_row = {'Algorithm':'Decision Tree', 'Accuracy':Tree_acc, 'Jaccard':Tree_jacc, 'F1-score':Tree_f1}
report = report.append(Tree_row, ignore_index=True)

log_row = {'Algorithm':'SVM', 'Accuracy':svm_acc, 'Jaccard':svm_jacc, 'F1-score':svm_f1 }
report = report.append(log_row, ignore_index=True)

svm_row = {'Algorithm':'LogisticRegression', 'Accuracy':log_acc,'Jaccard':log_jacc, 'F1-score':log_f1}
report = report.append(svm_row, ignore_index=True)
report

,Algorithm,Accuracy,Jaccard,F1-score
0,KNN,0.998973,0.998973,0.998973
1,Decision Tree,1.000000,1.000000,1.000000
2,SVM,0.999486,0.999486,0.999487
3,LogisticRegression,1.000000,1.000000,1.000000
